In [ ]:
#!pip install Google-API-Python-Client
#!pip install emoji

In [5]:
from googleapiclient.discovery import build
import re
import emoji

#API-Key zum laden der Daten
Google_API_KEY = '---'

#Auswahl der API
ytApi = build('youtube', 'v3', developerKey=Google_API_KEY)

#Eingabe der Video ID des Filmtrailers
videoId = 'd9MyW72ELq0'
print('Video-ID..: ' + videoId)

requestResponse = ytApi.videos().list(
    part='snippet',
    id=videoId
).execute()

requestSnippet = requestResponse['items'][0]['snippet']
idUploaderChannel = requestSnippet['channelId']

#250 Kommentare laden
print("Fetch comments...")
allComments = []
nextPageToken = None
while len(allComments) < 250: 
    request = ytApi.commentThreads().list(
        part='snippet',
        videoId=videoId,
        maxResults=100, 
        pageToken=nextPageToken
    )
    response = request.execute()
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        if comment['authorChannelId']['value'] != idUploaderChannel:
            allComments.append(comment['textDisplay'])
    nextPageToken = response.get('nextPageToken')

    if not nextPageToken:
        break
print(allComments[:10])

f = open("YTCommentsAvatar2.csv", 'w', encoding='utf-8')
for iD, commentAllCsv in enumerate(allComments):
    f.write(str(commentAllCsv)+'\n')
f.close()
print("Save comments..")

#Hyperlinks aus den Kommentaren herauslöschen
patternHyperlink = re.compile(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

thresholdEmojiRatio = 1.00

filteredComments = []

#Den Filter anwenden
for textFromComment in allComments:

    textFromComment = textFromComment.lower().strip()

    emojis = emoji.emoji_count(textFromComment)
    
#Nur Text im Ascii-Format herunterladen
    text_characters = len(re.sub(r'\s', '', textFromComment))
    if (any(char.isalnum() for char in textFromComment)) and not patternHyperlink.search(textFromComment) and textFromComment.isascii():
        if emojis == 0 or (text_characters / (text_characters + emojis)) > thresholdEmojiRatio:
            filteredComments.append(textFromComment)
            
#Gebe 10 Beispielkommentare aus
print(filteredComments[:10])

#Speichere die Datei als CSV ab
f = open("YTQACommentsAvatar2.csv", 'w', encoding='utf-8')
for iD, filteredCommentCsv in enumerate(filteredComments):
    f.write(str(filteredCommentCsv)+"\n")
f.close()
print("Relevant comments saved..")

Video-ID..: d9MyW72ELq0
Fetch comments...
['I just swear to god, if could live peacefully in the way of water, I would. I would even change my name. Just imagine having such life, such places to live in.', 'SEED... Energy... Bow at man... Lemurian... Man and AVATAR?', '<a href="https://www.youtube.com/watch?v=d9MyW72ELq0&amp;t=26">0:26</a> enuf to make the movie blockbuster', 'Jai sudheer anna', 'Jai sudheer anna', 'Avatar means form... Pandora creatures... 99% don&#39;t know the meaning of Avtar... Yhey aren&#39;t avtar... Avtar is noun describing forns', 'IT ENDED OK,INPROSSIBLE TRY WAY.', 'AS INC. RULES....', 'Jai sudeer anna 🔥🔥', '<a href="https://www.youtube.com/watch?v=d9MyW72ELq0&amp;t=68">1:08</a>']
Save comments..
['i just swear to god, if could live peacefully in the way of water, i would. i would even change my name. just imagine having such life, such places to live in.', 'seed... energy... bow at man... lemurian... man and avatar?', 'jai sudheer anna', 'jai sudheer anna', 